In [1]:
import tensorflow as tf

from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add, Flatten, Dense, Input 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from keras_tuner import HyperModel, RandomSearch # hyperparameter tuning için


### 1)Veri Setinin Yüklenmesi

In [2]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

### 2)Veri Setinin Boyut Düzenlemesi ve Normalizasyonu

In [3]:
# veri setinin reshape yapalim (28x28) -> (28x28x1), normalizasyon (/255)
train_images = train_images.reshape(-1, 28, 28, 1).astype("float32") / 255.0
test_images = test_images.reshape(-1, 28, 28, 1).astype("float32") / 255.0

### 3)Etiketleri One-Hot Encoding ile Düzenlenmesi

In [4]:
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

### 4)Residual Block Hazırlanılması

In [6]:
def residual_block(x, filters, kernel_size = 3, stride = 1):
    
    shortcut = x
    
    # 1. Conv katmanı
    x = Conv2D(filters, kernel_size = kernel_size, strides = stride, padding = "same")(x)
    x = BatchNormalization()(x) # Normalizasyon işlemi
    x = Activation("relu")(x) # Aktivasyon fonksiyonu
    
    # 2. conv katmani
    x = Conv2D(filters, kernel_size = kernel_size, strides = stride, padding = "same")(x)
    x = BatchNormalization()(x) 
    
    # eger giristen gelen verinin boyutu filtre sayısına eşit değilse
    if shortcut.shape[-1] != filters:
        # Giriş verisinin boyutunu eşitlemek icin 1x1 konvolusyon işlemi
        shortcut = Conv2D(filters, kernel_size=1, strides = stride, padding = "same")(shortcut)
        shortcut = BatchNormalization()(shortcut)
    
    # Residual Bağlantı : Giriş verisi ile çikiş verisini toplayalım
    x = Add()([x, shortcut])
    x = Activation("relu")(x)
    
    return x
    

### 5)Model Create - Compile

In [7]:
class ResNetModel(HyperModel):
    
    def build(self, hp): # hp = hyper parameter tuning için kullanılacak parametre
    
        inputs = Input(shape=(28,28,1))

        # 1. Conv layer
        x = Conv2D(filters = hp.Int("initial_filters", min_value = 32, max_value = 128, step = 32),
                   kernel_size = 3, padding = "same", activation = "relu")(inputs)
        x = BatchNormalization()(x)
        
        # Residual Blok ekleyelim
        for i in range(hp.Int("num_blocks", min_value = 1, max_value = 3, step = 1)):
            x = residual_block(x, hp.Int("res_filters_" + str(i), min_value = 32, max_value = 128, step =32))
        
        # Sınıflandirma katmanı
        x = Flatten()(x)
        x = Dense(128, activation = "relu")(x)
        outputs = Dense(10, activation = "softmax")(x)
        
        model = Model(inputs, outputs)
        
        # Compile Model
        model.compile(optimizer = Adam(hp.Float("learning_rate", min_value = 1e-4, max_value = 1e-2, sampling = "LOG")),
                      loss = "categorical_crossentropy",
                      metrics = ["accuracy"])
        return model  
    


In [8]:
tuner = RandomSearch(
    ResNetModel(),
    objective = "val_accuracy", # tuning referans degeri
    max_trials = 2, # toplam deneme sayisi
    executions_per_trial = 1, # her denemede kac kere egitim yapilacagi
    directory = "resnet_hyperparameter_tuning_directory",
    project_name = "resnet_model_tuning"
    )

In [9]:
# Hiperparametre optimizasyonu and training
tuner.search(train_images, train_labels, epochs = 1, validation_data = (test_images, test_labels))

Trial 2 Complete [00h 07m 14s]
val_accuracy: 0.8188999891281128

Best val_accuracy So Far: 0.8427000045776367
Total elapsed time: 00h 09m 51s


### 6)Modelin Test Edilmesi

In [10]:
#En iyi Modelin Alınması
best_model = tuner.get_best_models(num_models=1)[0]

# Test Seti ile en iyi modeli test edelim
test_loss, test_acc = best_model.evaluate(test_images, test_labels)
print(f"Test loss: {test_loss:.4f}, test accuracy: {test_acc:.4f}")

C:\Users\emirh\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 58 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.8443 - loss: 0.4495
Test loss: 0.4570, test accuracy: 0.8427
